# Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import math
from prettytable import PrettyTable

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ais-datasets/diabetes.csv")
data

# Предварительная обработка

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

Отсутствующих значений не обнаружено

# Разделение датасета на матрицу признаков `X` и вектор зависимых переменных `Y`

In [ ]:
X = data.drop('Outcome', axis=1)
Y = data['Outcome']

In [ ]:
X

In [ ]:
Y

# Кодирование категориальных признаков

Не требуется:

In [ ]:
X

# Разделение данных на обучающую и тестовую выборки

In [ ]:
# Список индексов данных
indices = list(range(len(X)))

# Размер тестовой выборки
test_size = 0.2 # 20%

# Начальное состояние генератора случайных чисел
random.seed(42)

# Шафлим данные (чтобы потом не балансировать)
random.shuffle(indices)

split_index = int(len(X) * test_size)

X_train = X.iloc[indices[split_index:]]
X_test = X.iloc[indices[:split_index]]
Y_train = Y.iloc[indices[split_index:]]
Y_test = Y.iloc[indices[:split_index]]

Данные после разделения:

In [ ]:
print('X_train ->', X_train.shape)
print('X_test ->', X_test.shape)
print('Y_train ->', Y_train.shape)
print('Y_test ->', Y_test.shape)

# Масштабирование данных

## Min-max scaler

Для обучающей выборки

In [ ]:
for column_name, params in X_train.items():
  minimum = min(params)
  maximum = max(params)
  difference = maximum - minimum
  X_train[column_name] = (X_train[column_name] - minimum) / difference

Для тестовой выборки

In [ ]:
for column_name, params in X_test.items():
  minimum = min(params)
  maximum = max(params)
  difference = maximum - minimum
  X_test[column_name] = (X_test[column_name] - minimum) / difference

Данные после масштабирования

In [ ]:
X_train.head()

In [ ]:
X_test.head()

# Реализация метода логистической регрессии

## Определим сигмоидную функцию и функцию потерь

In [ ]:
def sig(t):
    return 1 / (1 + np.exp(-t))

In [ ]:
def cost(Y_actual, Y_predicted):
    return -np.mean(Y_actual * np.log(Y_predicted) + (1 - Y_actual) * np.log(1 - Y_predicted))

## Определим методы обучения

### Градиентный спуск

In [ ]:
def gradient_descent(X_train, Y_train, iterations, learning_rate):
    objects_num, characteristics_num = X_train.shape

    weights = np.zeros(characteristics_num)
    losses = []
    bias = 0

    for iteration in range(1, iterations + 1):

        t = np.dot(X_train, weights) + bias
        #  prediction
        z = sig(t)

        #  ЧП стоимости по весам
        dw = (1 / objects_num) * np.dot(X_train.T, (z - Y_train))
        #  ЧП стоимости по смещению
        db = (1 / objects_num) * np.sum(z - Y_train)

        weights -= learning_rate * dw
        bias -= learning_rate * db

        if iteration % 100 == 0:
            loss = cost(Y_train, z)
            losses.append(loss)
            # print(f'{iteration}) cost = {loss}')

    coeff = {'weights': weights, 'bias': bias}
    return coeff, losses

# gradient_descent(X_train, Y_train, 100, 0.01)

### Оптимизация Ньютона

In [ ]:
def newton_optimization(X_train, Y_train, iterations):
    objects_num, characteristics_num = X_train.shape

    weights = np.zeros(characteristics_num)
    losses = []
    bias = 0

    for iteration in range(1, iterations + 1):

        t = np.dot(X_train, weights) + bias
        #  prediction
        z = sig(t)

        #  ЧП стоимости по весам
        dw = (1 / objects_num) * np.dot(X_train.T, (z - Y_train))
        #  ЧП стоимости по смещению
        db = (1 / objects_num) * np.sum(z - Y_train)

        hessian = (1 / objects_num) * (X_train.T @ ((z * (1 - z)) * X_train.T).T)

        weights -= np.linalg.inv(hessian) @ dw
        bias -= db

        if iteration % 100 == 0:
            loss = cost(Y_train, z)
            losses.append(loss)
            # print(f'{iteration}) cost = {loss}')

    coeff = {'weights': weights, 'bias': bias}
    return coeff, losses

# newton_optimization(X_train, Y_train, 100, 0.01)

## Определим функцию предсказания

In [ ]:
def predict(X_test, coeff):
    weights = coeff['weights']
    bias = coeff['bias']

    t = np.dot(X_test, weights) + bias

    z = sig(t)

    return (z > 0.6).astype(int)

# coeff, losses = newton_optimization(X_train, Y_train, 100, 0.01)
# predict(X_train, coeff)

# Оценка модели

Определим функцию для подсчета метрик

In [ ]:
def calculate_metrics(Y_prediction, Y_test):
    TP = np.sum((Y_prediction == 1) & (Y_test == 1))
    TN = np.sum((Y_prediction == 0) & (Y_test == 0))
    FP = np.sum((Y_prediction == 1) & (Y_test == 0))
    FN = np.sum((Y_prediction == 0) & (Y_test == 1))

    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

    return {'accuracy': accuracy, 'precision': precision,  'recall': recall, 'f1_score': f1_score}

# Исследование гиперпараметров

Создадим вариации гиперпараметров

In [ ]:
rates = [0.01, 0.2, 0.375, 0.5]
iterations = [100, 1000, 5000]

In [ ]:
max_f1_score = 0
best_params = {}
table = PrettyTable(['method', 'rate', 'iterations', 'accuracy', 'precision', 'recall', 'f1_score', 'losses'])
table.align['rate'] = "l"
table.align['iterations'] = "l"
table.align['accuracy'] = "l"
table.align['precision'] = "l"
table.align['recall'] = "l"
table.align['f1_score'] = "l"
table.align['losses'] = "l"

#  Для метода градиентного спуска
for rate in rates:
    for iteration in iterations:

        coeff, losses = gradient_descent(X_train, Y_train, iteration, rate)
        Y_prediction = predict(X_test, coeff)

        metrics = calculate_metrics(Y_prediction, Y_test)

        if (metrics['f1_score'] > max_f1_score):
            best_params = {'method': gradient_descent.__name__, 'rate': rate, 'iterations': iteration, 'accuracy': metrics['accuracy'], 'precision': metrics['precision'], 'recall': metrics['recall'], 'f1_score': metrics['f1_score'], 'losses': losses[0] - losses[len(losses) - 1]}

        table.add_row([gradient_descent.__name__, rate, iteration, metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1_score'], losses[0] - losses[len(losses) - 1]])

#  Для метода Ньютона
for iteration in iterations:

    coeff, losses = newton_optimization(X_train, Y_train, iteration)
    Y_prediction = predict(X_test, coeff)

    metrics = calculate_metrics(Y_prediction, Y_test)

    if (metrics['f1_score'] > max_f1_score):
        best_params = {'method': newton_optimization.__name__, 'rate': '-', 'iterations': iteration, 'accuracy': metrics['accuracy'], 'precision': metrics['precision'], 'recall': metrics['recall'], 'f1_score': metrics['f1_score'], 'losses': losses[0] - losses[len(losses) - 1]}

    table.add_row([newton_optimization.__name__, '-', iteration, metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1_score'], losses[0] - losses[len(losses) - 1]])

print(table)

Выведем лучшую калибровку гиперпараметров

In [ ]:
best_params_table = PrettyTable(['method', 'rate', 'iterations', 'accuracy', 'precision', 'recall', 'f1_score', 'losses'])
best_params_table.add_row([best_params['method'], best_params['rate'], best_params['iterations'], best_params['accuracy'], best_params['precision'], best_params['recall'], best_params['f1_score'], best_params['losses']])

print(best_params_table)

# Выводы

В ходе анализа представленной таблицы сравнения был сделан следующий вывод:

- Метод ньютона в среднем работает точнее
- Для выбора оптимального количества итераций методу ньютона нет надобности варьировать значение learning_rate (шаг) в связи с вычислением гессиана (второй производной функции)
- Метод градиентного спуска не всегда сходится к оптимальному решению  (из-за сложной формы функции и множества локальных минимумов)